In [1]:
# libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
import requests
import numpy as np
import tqdm
import os

In [2]:
## Initiating the browser
options = webdriver.ChromeOptions()
path = "../chrome\chromedriver.exe"
service = Service(executable_path=path)
browser = webdriver.Chrome(service=service,options=options)
browser.maximize_window()

browser.set_page_load_timeout(60) # only for 5 minutes


In [3]:
def getEmailFromWebsite(url):
    global browser 
    
    emails = set()
    # opening with selnium
    browser.get(url)
    
    # from any element which has @ in their text
    elems = browser.find_elements("xpath","//*[contains(text(), '@')]")
    for elem in elems:
        es = ((re.findall(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b", elem.text, re.I)))
        emails.update(es)
    
    if len(emails) == 0:
        try:
            
            time.sleep(1)
            button = browser.find_element(By.XPATH, "//a[contains(text(),'contact')]") 
            browser.execute_script("arguments[0].click();", button)


            elems = browser.find_elements("xpath","//*[contains(text(), '@')]")
            for elem in elems:
                es = ((re.findall(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b", elem.text, re.I)))
                emails.update(es)
        except:
            pass
    
    return ";".join(emails)


In [94]:
url = "https://safapharm.com"
getEmailFromWebsite(url)

'info@safapharm.com'

In [4]:
def extractWebsiteFromName(name):
    global browser
    global country
    url_base = "https://www.google.com/search?q="
    url = url_base + name + " " + country

    # opening a url:
    browser.get(url)
    elems = browser.find_elements("xpath","//a[@jsname='UWckNb']")

    # check first
    elem = elems[0]
    link = elem.text.split("\n")[2]
    
    marketplaces = [
        'justdail',
        'amazon',
        'cphi',
        'dnb',
        'wikipedia',
        'linkedin',
        'indiamart'
    ]
    
    for m in marketplaces:
        if m in link:
            return None
    
    if 'http' in link:
        if "›" in link:
            link = link.split("›")[0].strip()

        for m in marketplaces:
            if m in link:
                return None
            
        return link
    else:
        # check second
        elem = elems[1]
        link = elem.text.split("\n")[2]
        if 'http' in link:
            if "›" in link:
                link = link.split("›")[0].strip()
            return link
        else:
            # check third
            elem = elems[1]
            link = elem.text.split("\n")[2]
            if 'http' in link:
                if "›" in link:
                    link = link.split("›")[0].strip()
                return link
    return None



In [78]:
name = "Julphar Iraq"
links = extractWebsiteFromName(name)
links

'https://www.julphar.net'

In [5]:
def extractEmail(row):
    
    name = row['name']
    website = row['website']
    
    emails_all = []
    
    if type(website) != float:
        # extract emails from website:
        emails = getEmailFromWebsite(website)
        emails_all.extend(emails.split(";"))
        
    else:
        # try extracting website from company name
        website = extractWebsiteFromName(name)
        row['website']=website
        
        emails = getEmailFromWebsite(website)
        emails_all.extend(emails.split(";"))
    
    row['emails']=";".join(emails_all)
    return row
    


In [6]:
country = "Egypt"
files = os.listdir("../results/names/{}".format(country))
len(files)

103

In [7]:
os.mkdir("../results/emails/{}".format(country))

In [8]:
files[-2:]

['Zawiyat Razin_companies.csv', '`Izbat al Burj_companies.csv']

In [9]:

for ind, file in enumerate(files):
    print("Index ::",ind)
    print("Started for file ::",file)
    
    updated_rows = []
    failed_rows = []
    existing_rows_names = []

    df = pd.read_csv("../results/names/{}/{}".format(country,file))
    rows = df.to_dict('records')
    
    for row in tqdm.tqdm(rows):
        if row['name'] not in existing_rows_names:
            try:
                n = extractEmail(row)
                updated_rows.append(n)
            except:
                failed_rows.append(row)
                row['emails']=""
                updated_rows.append(row)
                pass
            
        existing_rows_names.append(row['name'])
    
    # saving
    df = pd.DataFrame(updated_rows)
    df.to_csv("../results/emails/{}/{}".format(country,file),index=None)

Index :: 0
Started for file :: Abjij_companies.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [03:34<00:00, 11.27s/it]


Index :: 1
Started for file :: Abnub_companies.csv


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:23<00:00, 17.90s/it]


Index :: 2
Started for file :: Abu al Matamir_companies.csv


EmptyDataError: No columns to parse from file

In [22]:
## When it is taking too much time for a certain website:

for ind, file in enumerate(files):
    print("Index ::",ind)
    print("Started for file ::",file)
    
    updated_rows = []
    failed_rows = []
    existing_rows_names = []

    df = pd.read_csv("../results/names/{}/{}".format(country,file))
    rows = df.to_dict('records')
    
    for row in tqdm.tqdm(rows[46:]):
        if row['name'] not in existing_rows_names:
            try:
                n = extractEmail(row)
                updated_rows.append(n)
            except:
                failed_rows.append(row)
                row['emails']=""
                updated_rows.append(row)
                pass
            
        
    
    

Index :: 0
Started for file :: companies_1.csv


100%|██████████████████████████████████████████████████████████████████████████████████| 73/73 [12:20<00:00, 10.15s/it]


In [24]:
# remember to update the index accordingly
updated_rows[0]

{'name': 'Austex Pharmaceutical PTE LTD',
 'phone': '062008 92433',
 'website': 'http://www.austexpharma.com/',
 'address': '81 UBI Avenue 4 #06-03 UB.ONE',
 'emails': 'contact@austexpharma.com'}

In [26]:
# saving
df = pd.DataFrame(updated_rows)
df.to_csv("../results/emails/{}/{}".format(country,file),index=None)